In [ ]:
# =============================================================================
#  SOKM_es_editor.py · v1.0 (single-shot, no CLI, no dictionary)
#  — Post-edits Spanish translations given English source lines in ONE request
#    - Preserves <br/> for subtitles (enforced by system prompt you provide)
#    - Reads system prompt from external .txt file
# =============================================================================
import os, json
from pathlib import Path
from typing import Tuple

import pandas as pd
from dotenv import load_dotenv

# ── LLM provider selection ---------------------------------------------------
# Set MODEL_PROVIDER = "openai" or "gemini"
MODEL_PROVIDER = "openai"
MODEL_NAME     = "gpt-5"          # e.g., "gpt-5" or "gemini-2.5-pro"

load_dotenv()  # loads OPENAI_API_KEY or GOOGLE_API_KEY if present

# Provider initialization (kept simple, no CLI)
def _init_provider(provider: str, model_name: str):
    p = provider.lower().strip()
    if p == "openai":
        import openai
        client = openai.OpenAI()
        return p, client, model_name
    elif p == "gemini":
        import google.generativeai as genai
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        return p, genai, model_name
    else:
        raise ValueError(f"Unknown provider: {provider!r}. Use 'openai' or 'gemini'.")

# ── Helpers ------------------------------------------------------------------
def read_file(path: str) -> str:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"File not found: {path}")
    return p.read_text(encoding="utf-8").strip()

def read_csv(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Input CSV not found: {path}")
    return pd.read_csv(p)

def find_columns(df: pd.DataFrame) -> Tuple[str, str]:
    """
    Detect English and Spanish columns:
      - English: 'Cleaned Text' (common in your file) or 'Segment Text'
      - Spanish: 'Translated Text'
    """
    candidates_en = ["Cleaned Text", "Segment Text"]
    en_col = next((c for c in candidates_en if c in df.columns), None)
    es_col = "Translated Text" if "Translated Text" in df.columns else None
    if not en_col or not es_col:
        raise ValueError(
            "CSV must include English + Spanish columns. "
            "English: 'Cleaned Text' or 'Segment Text' | Spanish: 'Translated Text'"
        )
    return en_col, es_col

def build_pairs(df: pd.DataFrame, en_col: str, es_col: str) -> dict:
    """
    Build {row_index_str: {"en": ..., "es": ...}} for all rows with strings on both sides.
    """
    pairs = {}
    for idx, row in df.iterrows():
        en = row.get(en_col, None)
        es = row.get(es_col, None)
        if isinstance(en, str) and isinstance(es, str):
            pairs[str(idx)] = {"en": en, "es": es}
    return pairs

def build_user_prompt(pairs: dict, provider: str) -> str:
    """
    User prompt; your system prompt (with <br/> preservation rules) is read from file.
    """
    pairs_json = json.dumps(pairs, ensure_ascii=False, indent=2)
    if provider == "openai":
        return (
            "# Correction Task\n\n"
            "You will receive a JSON object where each key maps to an object: "
            '{"en": "...", "es": "..."}. '
            "Return ONE JSON object with the SAME KEYS and the corrected Spanish string as each value. "
            "Keep strings unchanged when already correct. Follow the system prompt rules.\n\n"
            "## Input JSON\n" + pairs_json
        )
    else:
        return (
            "Correct the Spanish lines in the JSON (same keys). Return JSON only.\n\n"
            "Input JSON:\n" + pairs_json
        )

def call_llm_one_shot(provider: str, client, model_name: str, system_prompt: str, pairs: dict) -> dict:
    """
    Make a single LLM call with the entire transcript. Returns corrections dict.
    """
    user_prompt = build_user_prompt(pairs, provider)
    try:
        if provider == "openai":
            # Requires OPENAI_API_KEY in env
            resp = client.chat.completions.create(
                model=model_name,
                temperature=1,
                response_format={"type": "json_object"},
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": user_prompt},
                ],
            )
            return json.loads(resp.choices[0].message.content)

        else:  # gemini
            # Requires GOOGLE_API_KEY in env
            model = client.GenerativeModel(
                model_name=model_name,
                system_instruction=system_prompt,
                generation_config={
                    "temperature": 1,
                    "response_mime_type": "application/json",
                },
            )
            resp = model.generate_content(user_prompt)
            return json.loads(resp.text)

    except Exception as e:
        print("⚠️  LLM call failed:", e)
        return {}

def apply_corrections(df: pd.DataFrame, corrections: dict, output_col: str) -> pd.DataFrame:
    df[output_col] = pd.NA
    for k, v in corrections.items():
        try:
            i = int(k)
        except ValueError:
            continue
        df.loc[i, output_col] = v
    return df

# ── Main (everything configured here) ---------------------------------------
if __name__ == "__main__":
    # ‣‣‣ Adjust these paths --------------------------------------------------
    input_csv_path     = r"D:\SOKM\11 Identity 2 SoKM 2024 - 2025\11 Identity 2 SoKM 2024 - 2025_transcript_english_SE_br_converted_cleaned_gpt_4_1_translated_d_openai_gpt_5_pv2.0.csv"
    system_prompt_file = "system_prompt_editor_v1.0.txt"   # your post-edit rules (preserve <br/>)
    output_csv_path    = input_csv_path.replace(".csv", "_edited.csv")
    # ------------------------------------------------------------------------

    provider, client, model_name = _init_provider(MODEL_PROVIDER, MODEL_NAME)

    # Load resources
    system_prompt = read_file(system_prompt_file)
    df = read_csv(input_csv_path)
    en_col, es_col = find_columns(df)

    print(f"Detected English column:  {en_col}")
    print(f"Detected Spanish column:  {es_col}")

    # Build one-shot payload
    pairs = build_pairs(df, en_col, es_col)
    if not pairs:
        raise RuntimeError("No valid EN–ES pairs found to process.")

    print(f"Submitting {len(pairs)} lines in ONE request to {provider}:{model_name} …")
    print("Note: very large transcripts can hit model token limits.")

    # Single LLM call (no batching)
    corrections = call_llm_one_shot(
        provider=provider,
        client=client,
        model_name=model_name,
        system_prompt=system_prompt,
        pairs=pairs,
    )
    if not corrections:
        raise RuntimeError("Empty corrections received from model.")

    # Apply corrections, keep originals where missing
    corrected_col = "Translated Text (edited)"
    df = apply_corrections(df, corrections, corrected_col)
    mask_missing = df[corrected_col].isna()
    if mask_missing.any():
        df.loc[mask_missing, corrected_col] = df.loc[mask_missing, es_col]

    # Save
    Path(output_csv_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ Finished → {output_csv_path}")


In [ ]:
#verify if the entries in the column are identical between two DataFrames
#Any column with the same name needs to be verified
def verify_identical_entries(df1: pd.DataFrame, df2: pd.DataFrame) -> dict:
    common_columns = df1.columns.intersection(df2.columns)
    results = {}
    for col in common_columns:
        results[col] = df1[col].equals(df2[col])
    return results

if __name__ == "__main__":
    df1 = pd.read_csv(r"D:\SOKM\11 Identity 2 SoKM 2024 - 2025\11 Identity 2 SoKM 2024 - 2025_transcript_english_SE_br_converted_cleaned_gpt_4_1_translated_d_openai_gpt_5_pv2.0.csv")
    df2 = pd.read_csv(r"D:\SOKM\11 Identity 2 SoKM 2024 - 2025\11 Identity 2 SoKM 2024 - 2025_transcript_english_SE_br_converted_cleaned_gpt_4_1_translated_d_openai_gpt_5_pv2.0_edited.csv")
    results = verify_identical_entries(df1, df2)
    for col, is_identical in results.items():
        status = "identical" if is_identical else "different"
        print(f"Column '{col}' is {status} between the two DataFrames.")
        if not is_identical:
            differing_rows = df1[df1[col] != df2[col]]
            print(f"Differing rows in column '{col}':")
            print(differing_rows)
    print("Verification complete.")


In [ ]:
#Verify identical entries between two columns in the same csv
def verify_identical_columns(df: pd.DataFrame, col1: str, col2: str) -> bool:
    if col1 not in df.columns or col2 not in df.columns:
        raise ValueError(f"Columns '{col1}' and '{col2}' must be present in the DataFrame.")
    return df[col1].equals(df[col2])

if __name__ == "__main__":
    df = pd.read_csv(r"D:\SOKM\11 Identity 2 SoKM 2024 - 2025\11 Identity 2 SoKM 2024 - 2025_transcript_english_SE_br_converted_cleaned_gpt_4_1_translated_d_openai_gpt_5_pv2.0_edited.csv")

    col1 = "Translated Text"
    col2 = "Translated Text (edited)"
    are_identical = verify_identical_columns(df, col1, col2)
    if are_identical:
        print(f"Columns '{col1}' and '{col2}' are identical.")
    else:
        print(f"Columns '{col1}' and '{col2}' are different.")
        differing_rows = df[df[col1] != df[col2]]
        print(f"Number of differing rows: {len(differing_rows)}")
        print(f"Differing rows between columns '{col1}' and '{col2}':")
        print(differing_rows)
        print("Verification complete.")
